#### 메리츠화재 수수료 재산출

In [1]:
import pandas as pd
import shutil
import os

''' 실 디렉토리로 변경하여 사용~~~  '''
ym  = '202305'
dir0  = f'E:/글로벌백업(180329)/04.수수료작업/{ym}수수료/'
fileO = f'{ym}_수입원장(G).xlsx'  #원본
fileN = fileO.replace("(G)","")   #복사본
if os.path.isfile(dir0+fileN):
    os.remove(dir0+fileN)

shutil.copyfile(dir0+fileO, dir0+fileN)

DICT_RATE = {1:2.59, 7:0.12, 8:0.12, 9:0.12, 10:0.12, 11:0.12, 12:0.12}

df = pd.read_excel(dir0+fileN, sheet_name=['전체','메리츠화재'],dtype={'지급년월': 'object','계약일자': 'object', #'납입회차':'int64',
                '적용일':'object','FC코드':'object','납입기간':'object','영수일자':'object'})

In [2]:
df_m = df['메리츠화재']
df_temp = df_m[(df_m['계약종류']=='장기') & (df_m['계약일자']>='20230201') & (df_m['1차년성적']!=0) & (df_m['수수료계']!=0)]
df_src = df_m[~((df_m['계약종류']=='장기') & (df_m['계약일자']>='20230201') & (df_m['1차년성적']!=0) & (df_m['수수료계']!=0))]
print('메리츠건수 : 총 = ', len(df_temp)+len(df_src), '대상= ', len(df_temp), '비대상= ', len(df_src))

import math
print('이전 수수료계= ', sum(df_temp['수수료계']),' 신계약성과: ',sum(df_temp['장기신계약성과']),' 미유지환수: ',sum(df_temp['미유지환수']),
                            ' 장기모집: ',sum(df_temp['장기모집']))
df_new = pd.DataFrame()

for idx, row in df_temp.iterrows():
    if row['계약일자']:
        dd = row['계약일자']
    else:
        dd='00000000'
    pay_seq = row['납입회차']
    #     미유지환수 = 수수료계 => 환수
    #     수수료계 = 장기모집 + 계약관리 +신계약성과
    c_rate = DICT_RATE.get(pay_seq,0)  #해당회차 지급율
    #--환수
    if row['수수료계'] == row['미유지환수']:
        new_val = math.floor(row['미유지환수'] * 259 / 333)      
        row['미유지환수'] =  new_val
    elif row['수수료계'] == row['장기모집'] + row['장기계약관리'] + row['장기신계약성과']: 
        new_val = math.floor(row['1차년성적'] * c_rate) #새로운 수수료 = 환산 * new지급율
        if pay_seq==1:
            row['장기신계약성과'] = new_val
        else:
            row['장기계약관리'] = new_val
  
    new_sum = sum(row[33:48])
    
    # print(idx,' ',pay_seq,' ', row['수수료계'],'=>',new_sum ,' ',row['장기모집'],' ',row['장기신계약성과'],' ', row['미유지환수'])
    row['수수료계']   = new_sum
    df_new= pd.concat([row.to_frame().T,df_new])
    
print('이후 수수료계= ', sum(df_new['수수료계']),' 신계약성과: ',sum(df_new['장기신계약성과']),' 미유지환수: ',sum(df_new['미유지환수']),
                            ' 장기모집: ',sum(df_new['장기모집']))
print('차이= ', sum(df_new['수수료계']) - sum(df_temp['수수료계']))

메리츠건수 : 총 =  25046 대상=  213 비대상=  24833
이전 수수료계=  71668175  신계약성과:  71663209  미유지환수:  0  장기모집:  4966
이후 수수료계=  55692830  신계약성과:  55687864  미유지환수:  0  장기모집:  4966
차이=  -15975345


In [3]:
df_wrt1 = pd.concat([df_src,df_new])
print('메리츠건수(작업후) : 총 = ', len(df_wrt1))

df_a = df['전체']
df_src2 = df_a[df_a['보험사'] != '메리츠화재']
df_wrt2 = pd.concat( [df_src2, df_wrt1.iloc[:,:33] ])

with pd.ExcelWriter(dir0+fileN,mode="a",engine='openpyxl',if_sheet_exists='replace') as writer:  
    df_wrt1.to_excel(writer, sheet_name='메리츠화재',index=False)
    df_wrt2.to_excel(writer, sheet_name='전체',index=False)
    #서식지정 => 속도가 너무 느림 => 엑셀에서 작업할 것(아래 참조)
    # workbook = writer.book
    # worksheet = writer.sheets["메리츠화재"]
    # worksheet1 = writer.sheets["전체"]
    # format0 = workbook.add_format({'bold': True,'font_color':'red','font_size':10})
    # format1 = workbook.add_format({'font_name': '맑은 고딕','font_size':9})
    # worksheet.set_column(first_col=1, last_col=49, cell_format=format1)
   
    
print('전체시트(작업후) : 총 = ', len(df_wrt2))

메리츠건수(작업후) : 총 =  25046
전체시트(작업후) : 총 =  84765


#### 위 작업종료후 수수료 작업util.xlsm의 원장서식복사를 실행할 것